# 2022 Data Cleaning and Aggregation

In [ ]:
import pandas as pd
import numpy as np
import os
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import helpers
geolocator = Nominatim(user_agent="epra")
%load_ext dotenv
%dotenv

In [ ]:
BASE_URL = os.environ.get('BASE_URL')

### Load files
Due to different file formats. Each file downloaded is treated as am individual data source.
A lot of manual cleaning was done using Excel

In [ ]:
# Load all the data files
jan = pd.read_csv(f'{BASE_URL}/data/raw/22/15th-January-14th-February-2022.csv')
feb = pd.read_csv(f'{BASE_URL}/data/raw/22/15th-February-14th-March-2022_Website.csv')
march = pd.read_csv(f'{BASE_URL}/data/raw/22/15th-March-2022-to-April-2022.csv')
april = pd.read_csv(f'{BASE_URL}/data/raw/22/15th-April-2022-14th-May-2022.csv')
may = pd.read_csv(f'{BASE_URL}/data/raw/22/Pump-Prices11-15-May-14-June-2022_Website.csv')
june = pd.read_csv(f'{BASE_URL}/data/raw/22/Pump-Prices-15-June-14-July-2022_Website.csv')
july = pd.read_csv(f'{BASE_URL}/data/raw/22/Prices-july-to-August-2022.csv')

august = pd.read_csv(f'{BASE_URL}/data/raw/22/Prices-August-to-September-2022.csv')
sept = pd.read_csv(f'{BASE_URL}/data/raw/22/Prices-september-to-october-2022.csv')
oct1 = pd.read_csv(f'{BASE_URL}/data/raw/22/Prices-Oct-to-Nov-2022.csv')
nov = pd.read_csv(f'{BASE_URL}/data/raw/22/Prices-15th-Nov-14th-Dec-2022.csv')
dec = pd.read_csv(f'{BASE_URL}/data/raw/22/15th-December-2022-14th-January-2023.csv')


In [ ]:
"""Prepare data for the first half of the year since the data is in the same format"""
def prep_data_jan_jun(data_list):
    dfs = []
    for data in data_list:
        # remove whatespaces in columns
        df = helpers.remove_column_whitespace(data)

        # clean the date
        df['Price_Period'] = helpers.sanitize_date(df, 'Price_Period')
        # Fix typos
        df = helpers.rename_towns(df)
        # Get town coordinates
        df = helpers.get_town_coordinates(df, 'Town')
        #  drop towns without coords
        df = df.dropna()
        dfs.append(df)
    combined = pd.concat(dfs)
    return combined

"""Prepare data for the second half of the year since the data is in the same format"""
def prep_data_jul_dec(data_list):
    dfs = []
    for data in data_list:
        period = data.columns[1]

        data.rename(columns={'Unnamed: 0': 'Price_Period', f'{period}': 'Town',
                    'MAXIMUM PUMP PRICES': 'Super', 'Unnamed: 3': 'Diesel', 'Unnamed: 4': 'Kerosene'}, inplace=True)
        print(period)
        df = data.dropna()
        # df = data

        # set the period
        df['Price_Period'] = df['Price_Period'].apply(lambda s: f'{period}')

        # clean the date
        df['Price_Period'] = helpers.sanitize_date(df, 'Price_Period')
        # # Fix town typos
        df = helpers.rename_towns(df)
        # # Get town coordinates
        df = helpers.get_town_coordinates(df, 'Town')
        # #  drop towns without coords
        df = df.dropna()
        dfs.append(df)
    combined = pd.concat(dfs)
    return combined


In [ ]:
# Prepare data for the second part of the year
first_half_2022 = prep_data_jan_jun([jan, feb, march, april, may, june])
first_half_2022


In [ ]:
# Prepare data for the second part of the year
second_half_2022 = prep_data_jul_dec([july, august, sept, oct1, nov, dec])
second_half_2022.head()


In [ ]:
# Combine the dataframes to create the 2022 dataset
year_2022 = pd.concat([first_half_2022, second_half_2022])
year_2022.head()

In [ ]:
# write export it as a csv
year_2022.to_csv(f'{BASE_URL}/data/combined.csv',index=False)
